In [11]:
from jenga.tasks.openml import OpenMLBinaryClassificationTask, OpenMLMultiClassClassificationTask, OpenMLRegressionTask
from jenga.corruptions.generic import MissingValues
    
import pandas as pd
import numpy as np

from data_imputation_paper.imputation.simple import ModeImputer
from data_imputation_paper.imputation.ml import KNNImputer, ForestImputer
from data_imputation_paper.imputation.dl import AutoKerasImputer
from data_imputation_paper.imputation.generative import GAINImputer, VAEImputer
from data_imputation_paper.evaluation import Evaluator

## Make Deterministic

In [2]:
seed = 42

## Create example tasks

In [3]:
task = OpenMLMultiClassClassificationTask(openml_id=4552, seed=seed)

## Insert missing values using jenga

In [4]:
missing_values = [
    MissingValues(column='V2', fraction=0.5, na_value=np.nan, missingness='MCAR'),
    MissingValues(column='V4', fraction=0.5, na_value=np.nan, missingness='MCAR'),
    MissingValues(column='V15', fraction=0.5, na_value=np.nan, missingness='MCAR')
]

## Mode Imputation

In [5]:
arguments = {
    "seed": seed
}

%time Evaluator(task, missing_values, ModeImputer, arguments).evaluate(3).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     29.453297    30.742588
MSE   1363.771604  1466.736224
RMSE    36.928861    38.293394

             score metric
baseline  0.477325     F1
imputed   0.373818     F1


Target Column: V4
                train      test
F1_micro     0.828773  0.826855
F1_macro     0.453184  0.452604
F1_weighted  0.751178  0.748502

             score metric
baseline  0.477325     F1
imputed   0.377139     F1


Target Column: V15
                train      test
F1_micro     0.108120  0.106007
F1_macro     0.012172  0.011976
F1_weighted  0.021490  0.020404

             score metric
baseline  0.477325     F1
imputed   0.378331     F1


CPU times: user 19.3 s, sys: 390 ms, total: 19.7 s
Wall time: 22.8 s


## KNN imputation

In [6]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_neighbors": [3, 5]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_neighbors": [3, 5]
    }
}

%time Evaluator(task, missing_values, KNNImputer, arguments).evaluate(3).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     30.013386    31.590695
MSE   1431.884548  1543.234087
RMSE    37.839304    39.278294

             score metric
baseline  0.477325     F1
imputed   0.386212     F1


Target Column: V4
                train      test
F1_micro     0.936599  0.938163
F1_macro     0.881511  0.886950
F1_weighted  0.934655  0.936749

             score metric
baseline  0.477325     F1
imputed   0.368601     F1


Target Column: V15
                train      test
F1_micro     0.523978  0.518846
F1_macro     0.423429  0.408722
F1_weighted  0.516981  0.515318

             score metric
baseline  0.477325     F1
imputed   0.371613     F1


CPU times: user 28 s, sys: 1.81 s, total: 29.8 s
Wall time: 27.6 s


## Forest imputation

In [7]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_estimators": [50, 100]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_estimators": [50, 100]
    }
}

%time Evaluator(task, missing_values, ForestImputer, arguments).evaluate(3).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     28.024964    28.646884
MSE   1271.045521  1310.976730
RMSE    35.651637    36.187736

             score metric
baseline  0.477325     F1
imputed   0.361302     F1


Target Column: V4
                train      test
F1_micro     0.952780  0.954653
F1_macro     0.913050  0.914957
F1_weighted  0.951728  0.954151

             score metric
baseline  0.477325     F1
imputed   0.378834     F1


Target Column: V15
                train      test
F1_micro     0.559429  0.558893
F1_macro     0.492039  0.431023
F1_weighted  0.555064  0.553835

             score metric
baseline  0.477325     F1
imputed   0.363271     F1


CPU times: user 42.2 s, sys: 942 ms, total: 43.1 s
Wall time: 38.7 s


## AutoKeras imputation

In [8]:
arguments = {
    "seed": seed,
    'max_trials': 2,
    'tuner': 'greedy',
    'validation_split': 0.1,
    'epochs': 2
}

%time Evaluator(task, missing_values, AutoKerasImputer, arguments).evaluate(3).report()

Trial 2 Complete [00h 00m 02s]
val_accuracy: 0.018099548295140266

Best val_accuracy So Far: 0.06334841996431351
Total elapsed time: 00h 00m 04s
Epoch 1/2
71/71 [==============================] - 0s 1ms/step - loss: 7.7420 - accuracy: 0.0506
Epoch 2/2
71/71 [==============================] - 0s 1ms/step - loss: 7.3746 - accuracy: 0.0467


[2021-03-04 08:36:07,075 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde3763430> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:36:07,075 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde3763430> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:36:07,080 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde3754af0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:36:07,080 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde3754af0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


...

[2021-03-04 08:36:08,112 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffddfff61f0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:36:08,118 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde3707af0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:36:08,118 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde3707af0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:36:08,123 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde0a5a8b0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:36:08,123 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde0a5a8b0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     28.235073    2

## GAIN imputation

In [9]:
arguments = {
    "seed": seed,
    "hyperparameter_grid": {
        "gain": {
            "alpha": [80, 120],
            "hint_rate": [0.5, 0.9],
            "noise": [0.001, 0.1]
        },
        "training": {
            "batch_size": [64, 256],
            "epochs": [5, 15]
        }
    }
}

%time Evaluator(task, missing_values, GAINImputer, arguments).evaluate(3).report()

[W 2021-03-04 08:39:24,084] Trial 0 failed, because the objective function returned nan.
[W 2021-03-04 08:42:57,612] Trial 8 failed, because the objective function returned nan.
[W 2021-03-04 08:43:28,704] Trial 23 failed, because the objective function returned nan.
[W 2021-03-04 08:45:42,898] Trial 25 failed, because the objective function returned nan.


Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     30.717060    32.083624
MSE   1603.682711  1736.439692
RMSE    40.041092    41.667007

             score metric
baseline  0.477325     F1
imputed   0.395767     F1


Target Column: V4
                train      test
F1_micro     0.892468  0.888693
F1_macro     0.747795  0.732253
F1_weighted  0.875572  0.867921

             score metric
baseline  0.477325     F1
imputed   0.391569     F1


Target Column: V15
                train      test
F1_micro     0.059871  0.055948
F1_macro     0.027654  0.026719
F1_weighted  0.033262  0.031974

             score metric
baseline  0.477325     F1
imputed   0.357449     F1


CPU times: user 13min, sys: 2min 30s, total: 15min 30s
Wall time: 9min 49s


## VAE imputation

In [12]:
arguments = {
    "seed": seed,
    "hyperparameter_grid": {
        "training": {
            "batch_size": [64, 256],
            "epochs": [5, 15]
        },
        "optimizer": {
            "learning_rate": [0.0005],
            "beta_1": [0.9],
            "beta_2": [0.999],
            "epsilon": [1e-7],
            "amsgrad": [False]
        },
        "neural_architecture": {
            "latent_dim_rel_size": [0.1],
            "n_layers": [1, 2],
            "layer_1_rel_size": [0.5],
            "layer_2_rel_size": [0.25],
        },
    }
}

%time Evaluator(task, missing_values, VAEImputer, arguments).evaluate(3).report()

[2021-03-04 08:47:15,712 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde331d1f0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:47:15,712 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde331d1f0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:47:15,834 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde331d1f0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:47:15,834 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde331d1f0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:47:28,345 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde0599820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


...

[2021-03-04 08:48:44,893 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde1565280> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:48:57,879 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde0af3820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:48:57,879 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde0af3820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:48:58,006 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde0af3820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


[2021-03-04 08:48:58,006 - def_function.py:113  - WARNING - called_with_tracing]: 11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7ffde0af3820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     36.304583    3